__Spotify Assignment__

The aim of this assignment is to create and save a dataset containing information about every song in a given playlist by requesting data from Spotify's API.

Write the code to save the data about the title, the name of the album, the name of the artist, the duration, the track number, the release date, the popularity, the id, the number of available markets, the danceability, the energy, the loudness, the mode, the speechiness, the acousticness, the instrumentalness, the liveness, the valence and the tempo for all the tracks included in your chosen playlist, as well as the data about the total number of followers, the first listed genre, the popularity for the artists and whether the tracks are include in the top 10 in a dataframe called df and save this DataFrame to a .csv file called 'spotify.csv'. When creating the dataframe make sure the column names are exactly the same as those of the lists you ccreated in previous exercises to store the different values.

In [1]:
# Store ID and Key
client_id = "43068dccdde7404aa5dde50b01333507"
client_secret = "7d7455420b5640438a2191eda099e884"

In [2]:
# Get access token
import requests

# URL for token resource
auth_url = 'https://accounts.spotify.com/api/token'

# request body
params = {'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret}

# POST the request
auth_response = requests.post(auth_url, params).json()

access_token = auth_response['access_token']

headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [3]:
base_url = 'https://api.spotify.com/v1/'
audio_features_endpoint = base_url + "audio-features"

In [4]:
# Choose any song (I chose Creep by Radiohead)
track_id = "6b2oQwSGFkzsMtQruIWm2p"
params={"ids":track_id}
creep = requests.get(audio_features_endpoint, headers = headers, params = params).json()

In [5]:
# Choose any playlist (I used my favorite playlist to study to - Fresh & Chill :D)
playlist_id = "37i9dQZF1DX5CdVP4rz81C"

In [6]:
playlist_url = "https://api.spotify.com/v1/playlists/" + playlist_id + "/tracks"
params = {"playlist_id":playlist_id}

playlist_response = requests.get(playlist_url, headers = headers)
playlist = playlist_response.json()

In [7]:
title = []
album = []
artist = []
duration = []
track_number = []
release_date = []
track_popularity = []
track_id = []
n_available_markets = []

x = 0

for y in playlist['items']:
    title.append(str(playlist['items'][x]['track']['name']))
    album.append(str(playlist['items'][x]['track']['album']['name']))
    artist.append(str(playlist['items'][x]['track']['album']['artists'][0]['name']))
    duration.append(int(playlist['items'][x]['track']['duration_ms']))
    track_number.append(int(playlist['items'][x]['track']['track_number']))
    release_date.append(str(playlist['items'][x]['track']['album']['release_date']))
    track_popularity.append(int(playlist['items'][x]['track']['popularity']))
    track_id.append(str(playlist['items'][x]['track']['id']))
    n_available_markets.append(int(len(playlist['items'][x]['track']['available_markets'])))
    x = x + 1

In [8]:
params = {"ids" : ",".join(track_id)}
features = requests.get(audio_features_endpoint, params = params, headers = headers).json()

danceability = []
energy = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

x = 0

for y in features['audio_features']:
    danceability.append(float(features['audio_features'][x]['danceability']))
    energy.append(float(features['audio_features'][x]['energy']))
    loudness.append(float(features['audio_features'][x]['loudness']))
    mode.append(int(features['audio_features'][x]['mode']))
    speechiness.append(float(features['audio_features'][x]['speechiness']))
    acousticness.append(float(features['audio_features'][x]['acousticness']))
    instrumentalness.append(features['audio_features'][x]['instrumentalness'])
    liveness.append(float(features['audio_features'][x]['liveness']))
    valence.append(float(features['audio_features'][x]['valence']))
    tempo.append(float(features['audio_features'][x]['tempo']))
    x = x + 1

In [9]:
endpoint = 'artists'
url = base_url + endpoint

id_artist = []
v = 0

for y in playlist['items']:
    id_artist.append(playlist['items'][v]['track']['album']['artists'][0]['id'])
    v = v + 1

    
artist_followers1 = []
genre1 = []
artist_popularity1 = []


params = {"ids" : ",".join(id_artist[0:50])}
artist_info = requests.get(url, params = params, headers = headers).json()

x = 0

for y in artist_info['artists']:
    artist_followers1.append(int(artist_info['artists'][x]['followers']['total']))
    if len(artist_info['artists'][x]['genres']) != 0:
        genre1.append(str(artist_info['artists'][x]['genres'][0]))
    else:
        genre1.append(None)
    artist_popularity1.append(int(artist_info['artists'][x]['popularity']))
    x = x + 1
 

artist_followers2 = []
genre2 = []
artist_popularity2 = []

params = {"ids" : ",".join(id_artist[50:100])}
artist_info = requests.get(url, params = params, headers = headers).json()

x = 0

for y in artist_info['artists']:
    artist_followers2.append(int(artist_info['artists'][x]['followers']['total']))
    if len(artist_info['artists'][x]['genres']) != 0:
        genre2.append(str(artist_info['artists'][x]['genres'][0]))
    else:
        genre2.append(None)
    artist_popularity2.append(int(artist_info['artists'][x]['popularity']))
    x = x + 1


artist_followers = artist_followers1 + artist_followers2
genre = genre1 + genre2
artist_popularity = artist_popularity1 + artist_popularity2

In [10]:
unique_artist_id = []
v = 0

for y in playlist['items']:
    unique_artist_id.append(playlist['items'][v]['track']['album']['artists'][0]['id'])
    v = v + 1

unique_artist_id = set(unique_artist_id)
unique_artist_id = list(unique_artist_id)

In [11]:
endpoint = 'artists'
params = {"id" : unique_artist_id, "market" : "ES"}

list_for_all_artist_urls = []

g = 0
x = 0
y = 0

for u in unique_artist_id:
    list_for_all_artist_urls.append(base_url + endpoint + "/" + unique_artist_id[g] + "/top-tracks")
    g = g + 1

    
top10_list = []
top_tracks = {}

for b in unique_artist_id:
    toptracks_of_given_artist_url = requests.get(url = list_for_all_artist_urls[x],
                                                 params = params, headers = headers).json()
    for c in toptracks_of_given_artist_url['tracks']:
        if y < 10:
            top10_list.append(toptracks_of_given_artist_url['tracks'][y]['name'])
            y = y + 1
    top_tracks[str(unique_artist_id[x])] = top10_list
    x = x + 1
    y = 0
    top10_list = []

In [12]:
is_top = []

x = 0

for i in title:
    if title[x] in top_tracks[id_artist[x]]:
        is_top.append('yes')
    else:
        is_top.append('not')
    x = x + 1

In [13]:
import pandas as pd

data = {'title':title,'album':album,'artist':artist,'duration':duration,'track_number':track_number,
        'release_date':release_date,'track_popularity':track_popularity,'track_id':track_id,
        'n_available_markets':n_available_markets,'danceability':danceability,'energy':energy,
        'loudness':loudness,'mode':mode,'speechiness':speechiness,'acousticness':acousticness,
        'instrumentalness':instrumentalness,'liveness':liveness,'valence':valence,'tempo':tempo,
        'artist_followers':artist_followers,'genre':genre,'artist_popularity':artist_popularity,
        'is_top':is_top}

df = pd.DataFrame(data,columns=['title','album','artist','duration','track_number',
        'release_date','track_popularity','track_id',
        'n_available_markets','danceability','energy',
        'loudness','mode','speechiness','acousticness',
        'instrumentalness','liveness','valence','tempo',
        'artist_followers','genre','artist_popularity',
        'is_top'])

df

,title,album,artist,duration,track_number,release_date,track_popularity,track_id,n_available_markets,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_followers,genre,artist_popularity,is_top
0,Meant To Be,Meant To Be,Ber,163226,1,2021-10-06,69,136J2Jkg8sG7P5wilO9P1R,184,0.500,...,0.0469,0.8700,0.000000,0.125,0.405,173.794,35157,gen z singer-songwriter,56,yes
1,Fall in Love with You.,Fall in Love with You.,Montell Fish,132000,1,2021-11-05,71,31MQR9rOHRXyg4J01hRY1h,184,0.483,...,0.0537,0.9220,0.012500,0.112,0.146,73.984,191978,pittsburgh rap,66,yes
2,I GUESS I'M IN LOVE,I GUESS I'M IN LOVE,Clinton Kane,204093,1,2021-08-20,87,1is8gU4RVcN4J8xItxWoOY,184,0.422,...,0.0351,0.8770,0.000018,0.149,0.223,77.213,622262,alt z,76,yes
3,Older,Older,Alec Benjamin,164399,1,2021-09-17,73,4qeHV9T5N3S7KAKryZQp8s,184,0.552,...,0.0617,0.5850,0.000000,0.151,0.507,84.115,3086177,alt z,79,yes
4,see you later (ten years),see you later (ten years),Jenna Raine,188160,1,2021-09-24,78,5elofKuRmgZmCVImWT6JtE,184,0.584,...,0.0301,0.2470,0.000000,0.151,0.253,86.102,31149,alt z,63,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Love Is Gone - Acoustic,Love Is Gone (Acoustic),SLANDER,176971,1,2019-11-13,79,7F1CiKqrY44kh5cDqwHOnx,184,0.541,...,0.0303,0.8820,0.000002,0.160,0.359,139.644,326630,dubstep,72,yes
86,Closure,Closure,Hayd,196277,1,2021-04-01,43,0mKQl9k1EErmEPk11aWZEl,0,0.696,...,0.0458,0.9670,0.000000,0.292,0.350,124.168,138989,alt z,61,yes
87,Darling,"If I Can’t Have Love, I Want Power",Halsey,182000,7,2021-08-27,56,0ExjUQBJz9GXbk5BI4t2IB,184,0.541,...,0.0312,0.8420,0.000000,0.208,0.584,86.947,16421928,dance pop,86,not
88,Ruthless,Ruthless,Cannons,205312,1,2021-10-22,58,4okcOFaWiH7Zl22gIRIA7M,184,0.558,...,0.0390,0.0137,0.002790,0.102,0.402,127.888,108295,vapor soul,63,yes


In [14]:
# What is your playlist's most popular song?
x = 0
highest_popularity = 0

for u in df:
    if track_popularity[x] > highest_popularity:
        current_holder = x
        highest_popularity = track_popularity[x]
        most_popular = str(title[x])
    elif track_popularity[x] == highest_popularity:
        if artist_followers[x] > artist_followers[current_holder]:
            current_holder = x
            highest_popularity = track_popularity[x]
            most_popular_ = str(title[x]) 
        else:
            most_popular = str(title[x])
    x = x + 1
    
print("The most popular song in my playlist: ", most_popular, "(with Spotify ;)")

The most popular song in my playlist:  I GUESS I'M IN LOVE (with Spotify ;)
